In [1]:
#From https://github.com/COSIMA/cosima-recipes/blob/master/DocumentedExamples/TemperatureSalinityDiagrams.ipynb

In [2]:
import dask
import dask.array as da
import dask.dataframe as dd
from dask import delayed
import numpy as np
import xarray as xr
from distributed.diagnostics.progressbar import progress
import cosima_cookbook as cc
import bokeh.plotting as bp

import matplotlib.pyplot as plt
import pandas as pd

import datashader as ds
import datashader.transfer_functions as tf
import gsw
import scipy as scipy
from scipy import stats
from xhistogram.xarray import histogram

import holoviews as hv
import holoviews.operation.datashader as hd

In [3]:
from dask.distributed import Client
client = Client()
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/42151/status,
Dashboard: /proxy/42151/status,Workers: 5
Total threads: 20,Total memory: 125.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40717,Workers: 5
Dashboard: /proxy/42151/status,Total threads: 20
Started: Just now,Total memory: 125.20 GiB
Comm: tcp://127.0.0.1:33273,Total threads: 4
Dashboard: /proxy/40417/status,Memory: 25.04 GiB
Nanny: tcp://127.0.0.1:43511,


In [ ]:
master_session = cc.database.create_session()

control = '01deg_jra55v13_ryf9091'

# temperature of meltwater at the freezing point calculated with the in situ salinity:
Tfreezing_insituS = 'accessom2-GPC001'
# temperature of meltwater at the freezing point calculated with the 0 salinity:
Tfreezing_zeroS = 'accessom2-GPC002'
# temperature of meltwater equal to the minimum of the above two temperatures:
Tfreezing_min = 'accessom2-GPC003'

start_time='2154-12-01'
end_time='2154-12-31'

lat_slice  = slice(-80,-59)

# build database for these experiments:
session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

dir_list = ['/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC001/',
           '/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC002/',
           '/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC003/',]
cc.database.build_index(dir_list,basal_melt_session)


In [ ]:
temp = cc.querying.getvar(control, 'temp', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time)
salt = cc.querying.getvar(control, 'salt', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time) 

In [13]:
t_blocks = [x.reshape((-1, )) for x in temp.data.to_delayed().reshape((-1, ))]
s_blocks = [x.reshape((-1, )) for x in salt.data.to_delayed().reshape((-1, ))]
dfs = [delayed(pd.DataFrame)({'temp': t, 'salt': s}, copy=False) for t, s in zip(t_blocks, s_blocks)]

In [ ]:
df = dd.from_delayed(dfs)
cvs = ds.Canvas(plot_width=400, plot_height=400)#, x_range =(31, 36), y_range=(-2, 3))
agg = cvs.points(df, 'salt', 'temp')
img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='log')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/datashader/glyphs/points.py:127: RuntimeWarning: All-NaN slice encountered
  np.nanmin(df[self.x].values).item(),
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/datashader/glyphs/points.py:128: RuntimeWarning: All-NaN slice encountered
  np.nanmax(df[self.x].values).item(),
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/datashader/glyphs/points.py:129: RuntimeWarning: All-NaN slice encountered
  np.nanmin(df[self.y].values).item(),
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/datashader/glyphs/points.py:130: RuntimeWarning: All-NaN slice encountered
  np.nanmax(df[self.y].values).item()]]
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.04/lib/python3.9/site-packages/datashader/glyphs/points.py:127: RuntimeWarning: All-NaN slice encountered
  np.nanmin(df[self.x].values).item(),
/g/data/h

In [ ]:
hd.shade.cmap=["lightblue", "darkblue"]
hv.extension("bokeh", "matplotlib") 

In [ ]:
hv.output(backend="matplotlib")
# agg = ds.Canvas().points(df, 'x', 'y')
hd.shade(hv.Image(agg))